In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

import os
from pathlib import Path

from vivarium_helpers.utils import convert_to_categorical, constant_categorical, print_memory_usage, Timer

from vivarium_helpers.projects.alzheimers import loading, population
from vivarium_helpers.projects.alzheimers.population import RunType
from vivarium_helpers.projects.alzheimers.results import AlzheimersResultsProcessor

!date
!whoami
!pwd

Tue Feb 17 16:44:47 PST 2026
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/results_tables


# Record version of Vivarium Helpers

In [3]:
!conda list | grep vivarium

# packages in environment at /ihme/homes/ndbs/miniconda3/envs/vivarium_csu_alzheimers_artifact:
vivarium                  3.4.12                   pypi_0    pypi
vivarium-build-utils      2.0.4                    pypi_0    pypi
vivarium-cluster-tools    2.1.17                   pypi_0    pypi
vivarium-csu-alzheimers   0.1.dev18+g43e997c.d20250808          pypi_0    pypi
vivarium-dependencies     1.0.1                    pypi_0    pypi
vivarium-gbd-access       4.2.4                    pypi_0    pypi
vivarium-helpers          0.2.0                     dev_0    <develop>
vivarium-inputs           6.0.4                    pypi_0    pypi
vivarium-public-health    4.3.5                    pypi_0    pypi


In [4]:
!pip freeze | grep vivarium

vivarium==3.4.12
vivarium_build_utils==2.0.4
vivarium_cluster_tools==2.1.17
-e git+https://github.com/ihmeuw/vivarium_csu_alzheimers.git@263eb3d9b80b13735a7380bb892430eaba92603a#egg=vivarium_csu_alzheimers
vivarium_dependencies==1.0.1
vivarium_gbd_access==4.2.4
-e git+https://github.com/ihmeuw/vivarium_helpers.git@14726994615ddaac5513f19155799d6debf4ecb7#egg=vivarium_helpers
vivarium_inputs==6.0.4
vivarium_public_health==4.3.5


# Define data directories

In [5]:
# Project directory
project_dir = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/')

# Output directory to store final results for client
output_dir = Path(r"J:\Project\simulation_science\alzheimers\results_in_progress".replace('\\', '/').replace('J:', '/snfs1'))
print(output_dir.exists())

# Output from multistate life table
mslt_output_dir = output_dir.parent / 'results_11_14_2025_mslt'
print(mslt_output_dir.exists())
output_dir

True
True


PosixPath('/snfs1/Project/simulation_science/alzheimers/results_in_progress')

In [6]:
# # For testing: Run directory containing model 8.3 results for all
# # locations
# model_run_subdir = 'results/abie_consistent_model_test/united_states_of_america/2025_10_28_08_55_05/'

# # Results directory for model 8.3, for testing
# results_dirs = project_dir / model_run_subdir / 'results/'

# Artifact for models 10.0 - 12.2
artifact_model_number = '10.0'


In [ ]:
# Model 8.4 results (final runs for 10/31/25 intermediate results)
# Each batch run contains all locations, 100 random seeds, and 3 or 4 draws
model_run_dir_8_4 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec')

# Model 8.7 results (updated final run completed on 11/10/2025)
model_run_dir_8_7 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec')

# Model 12.2 results (updated final run completed on 02/14/2026)
model_run_dir_12_2 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2')

# Batch runs are stored in "timestamp" subdirectories of the model run
# directory.
# In model 12.2, the run directories are nested under two more levels of
# subfolders: batch number, then model_spec, then timestamp.

# Select a model and filter for testing
batch_run_dirs = loading.list_batch_run_directories(model_run_dir_12_2)#[:2]

batch_run_dirs

['/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch5/model_spec/2026_02_13_17_56_25',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch9/model_spec/2026_02_14_06_54_15',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch11/model_spec/2026_02_14_06_55_56',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch8/model_spec/2026_02_13_21_06_03',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch12/model_spec/2026_02_14_07_00_06',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch1/model_spec/2026_02_13_12_41_14',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch3/model_spec/2026_02_13_17_10_34',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch2/model_spec/2026_02_13_15_02_52',
 '/mnt

# Create an AlzheimersResultsProcessor object to process results

In [8]:
results = AlzheimersResultsProcessor('10.0', RunType.FINAL, batch_run_dirs=batch_run_dirs)

In [9]:
results.initial_simulation_population

2000000

In [10]:
results.locations

['United States of America',
 'Brazil',
 'China',
 'Germany',
 'Israel',
 'Japan',
 'Spain',
 'Sweden',
 'Taiwan (Province of China)',
 'United Kingdom']

In [11]:
results.draws

[35,
 187,
 41,
 123,
 248,
 122,
 183,
 103,
 161,
 50,
 24,
 172,
 235,
 245,
 69,
 204,
 219,
 63,
 199,
 6,
 249,
 7,
 236,
 1,
 211]

In [12]:
len(set(results.draws))

25

# Load population data and calculate model scale

In [13]:
with Timer():
    results.load_population_data(append_aggregates=True)
results.ops.describe(results.model_scale)

Elapsed time: 0:00:13.450910


,count,mean,std,min,25%,50%,75%,max
location,,,,,,,,
Brazil,25.0,0.645126,0.042517,0.544775,0.618288,0.636986,0.682593,0.718303
China,25.0,0.075453,0.003924,0.067459,0.072855,0.075736,0.077771,0.085044
Germany,25.0,0.548393,0.025209,0.498396,0.536829,0.546442,0.557773,0.604171
Israel,25.0,26.039578,2.496689,20.549962,24.481746,25.631266,28.245708,30.287894
Japan,25.0,0.354952,0.020849,0.308678,0.339599,0.355165,0.367604,0.399881
Spain,25.0,1.865326,0.083683,1.719458,1.822242,1.856133,1.887104,2.134511
Sweden,25.0,7.785869,0.522496,6.614222,7.413090,7.838525,8.131916,8.716418
Taiwan (Province of China),25.0,5.271920,0.276774,4.659120,5.108985,5.251648,5.484657,5.726114
United Kingdom,25.0,1.229875,0.047735,1.167644,1.185930,1.220243,1.267019,1.318293


In [14]:
results.person_time

,location,sex,event_year,input_draw,value,age_group
0,United States of America,Female,2025,35,1.134189e+07,60_to_64
1,United States of America,Female,2025,187,1.101609e+07,60_to_64
2,United States of America,Female,2025,41,1.099131e+07,60_to_64
3,United States of America,Female,2025,123,1.141446e+07,60_to_64
4,United States of America,Female,2025,248,1.116892e+07,60_to_64
...,...,...,...,...,...,...
911995,Spain,Both,2100,249,1.830983e+07,60_plus
911996,Sweden,Both,2100,249,3.882594e+06,60_plus
911997,Taiwan (Province of China),Both,2100,249,8.897937e+06,60_plus
911998,United Kingdom,Both,2100,249,2.338290e+07,60_plus


# DALYs and averted DALYs

In [15]:
filename = 'ylls'
short_name = filename

with Timer():
    results.load_measure_from_batch_runs(filename, short_name, filters=loading.FINAL_RESULTS_FILTERS[filename])

print(len(results.data[short_name]), 'rows')
results.data[short_name].head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025'), ('entity', '=', 'alzheimers_disease_state')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch5/model_spec/2026_02_13_17_56_25/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,ylls,baseline,Female,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,ylls,baseline,Male,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,ylls,bbbm_testing,Female,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,ylls,bbbm_testing,Male,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,ylls,bbbm_testing_and_treatment,Female,0.0,Japan


In [16]:
filename = 'ylds'
short_name = filename

with Timer():
    results.load_measure_from_batch_runs(filename, short_name, filters=loading.FINAL_RESULTS_FILTERS[filename])

print(len(results.data[short_name]), 'rows')
results.data[short_name].head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025'), ('entity', '==', 'alzheimers_disease_and_other_dementias')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch5/model_spec/2026_02_13_17_56_25/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/m

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,sub_entity,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,ylds,baseline,Female,alzheimers_blood_based_biomarker_state,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,ylds,baseline,Female,alzheimers_mild_cognitive_impairment_state,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,ylds,baseline,Female,alzheimers_disease_state,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,ylds,baseline,Male,alzheimers_blood_based_biomarker_state,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,ylds,baseline,Male,alzheimers_mild_cognitive_impairment_state,0.0,Japan


In [17]:
dalys = results.process_dalys(results.data.ylls, results.data.ylds)
print_memory_usage(dalys)
print(len(dalys), 'rows')
dalys.tail()

97.479405 MB measure
46.179273 MB minuend
92.349273 MB subtrahend
32.499994 MB minuend re-indexed
64.989994 MB subtrahend re-indexed
64.990054 MB difference
64.989405 MB difference with reset index
68.40973 MB final difference
179.560067 MB 
8550000 rows


,age_group,artifact_path,entity,entity_type,event_year,measure,sex,sub_entity,input_draw,scenario,location,value,metric,disease_stage
8549995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,AD,cause,2100,Averted DALYs Associated with AD,Female,alzheimers_disease_state,211,bbbm_testing_and_treatment,Brazil,-4268.619345,Number,alzheimers_disease_state
8549996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,AD,cause,2100,Averted DALYs Associated with AD,Female,alzheimers_mild_cognitive_impairment_state,211,bbbm_testing_and_treatment,Brazil,-28.612690,Number,alzheimers_mild_cognitive_impairment_state
8549997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,AD,cause,2100,Averted DALYs Associated with AD,Male,alzheimers_blood_based_biomarker_state,211,bbbm_testing_and_treatment,Brazil,0.000000,Number,alzheimers_blood_based_biomarker_state
8549998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,AD,cause,2100,Averted DALYs Associated with AD,Male,alzheimers_disease_state,211,bbbm_testing_and_treatment,Brazil,-1445.694007,Number,alzheimers_disease_state
8549999,95_plus,/mnt/team/simulation_science/pub/models/vivari...,AD,cause,2100,Averted DALYs Associated with AD,Male,alzheimers_mild_cognitive_impairment_state,211,bbbm_testing_and_treatment,Brazil,-8.035070,Number,alzheimers_mild_cognitive_impairment_state


In [18]:
# 5m 22s to run on first 3 batches of 8.4 (10 draws)
with Timer():
    dalys_output = results.summarize_and_beautify(dalys)
print_memory_usage(dalys_output)
print(len(dalys_output), 'rows')
dalys_output.tail()

Timer SummarizingTimer started
2026-02-17 16:48:55.649405


2026-02-17 16:49:59.636192
2026-02-17 17:05:27.351911
Elapsed time: 991.7376 seconds
Elapsed time: 0:16:31.737752
35.027663 MB 
1094400 rows


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
1094395,2100,United States of America,60_plus,Male,MCI due to AD,BBBM Testing Only,Averted DALYs Associated with AD,Rate,1.895345e-22,-1.835786e-21,3.731131e-21
1094396,2100,United States of America,60_plus,Both,MCI due to AD,BBBM Testing Only,Averted DALYs Associated with AD,Rate,8.733336e-23,-8.379646e-22,1.711298e-21
1094397,2100,United States of America,60_plus,Female,MCI due to AD,BBBM Testing and Treatment,Averted DALYs Associated with AD,Rate,1.585297e-05,7.859840e-06,2.803107e-05
1094398,2100,United States of America,60_plus,Male,MCI due to AD,BBBM Testing and Treatment,Averted DALYs Associated with AD,Rate,1.085868e-05,4.940373e-06,1.765376e-05
1094399,2100,United States of America,60_plus,Both,MCI due to AD,BBBM Testing and Treatment,Averted DALYs Associated with AD,Rate,1.356095e-05,6.528566e-06,2.340380e-05


In [19]:
dalys_output.to_csv(output_dir / 'dalys.csv', index=False)

# Incidence

In [20]:
filename = 'counts_new_simulants'
short_name = 'incidence_bbbm'

with Timer():
    results.load_measure_from_batch_runs(filename, short_name, filters=loading.FINAL_RESULTS_FILTERS[filename])

print(len(results.data[short_name]), 'rows')
results.data[short_name].head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch5/model_spec/2026_02_13_17_56_25/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/spain.hdf'

,age_group,artifact_path,event_year,input_draw,scenario,sex,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,35,baseline,Female,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,35,baseline,Male,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,35,bbbm_testing,Female,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,35,bbbm_testing,Male,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,35,bbbm_testing_and_treatment,Female,0.0,Japan


In [21]:
filename = 'transition_count_alzheimers_disease_and_other_dementias'
short_name = 'transitions_ad'

with Timer():
    results.load_measure_from_batch_runs(filename, short_name, filters=loading.FINAL_RESULTS_FILTERS[filename])

print(len(results.data[short_name]), 'rows')
results.data[short_name].head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch5/model_spec/2026_02_13_17_56_25/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/spain.hdf'

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,sub_entity,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,transition_count,baseline,Female,alzheimers_blood_based_biomarker_state_to_alzh...,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,transition_count,baseline,Female,alzheimers_mild_cognitive_impairment_state_to_...,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,transition_count,baseline,Male,alzheimers_blood_based_biomarker_state_to_alzh...,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,transition_count,baseline,Male,alzheimers_mild_cognitive_impairment_state_to_...,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,transition_count,bbbm_testing,Female,alzheimers_blood_based_biomarker_state_to_alzh...,0.0,Japan


In [22]:
incidence_counts = results.process_incidence(
    results.data.incidence_bbbm, results.data.transitions_ad)
print(len(incidence_counts), 'rows')
print_memory_usage(incidence_counts)
incidence_counts

71.825977 MB raw transitions
71.825977 MB after marginalizing treatment
711.365977 MB after state extraction
['alzheimers_mild_cognitive_impairment_state' 'alzheimers_disease_state']
5130000 rows
138.519494 MB 


,age_group,artifact_path,event_year,input_draw,scenario,sex,value,location,to_state,measure,metric,disease_stage
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,35,baseline,Female,0.0,Japan,alzheimers_blood_based_biomarker_state,Incidence,Number,alzheimers_blood_based_biomarker_state
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,35,baseline,Male,0.0,Japan,alzheimers_blood_based_biomarker_state,Incidence,Number,alzheimers_blood_based_biomarker_state
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,35,bbbm_testing,Female,0.0,Japan,alzheimers_blood_based_biomarker_state,Incidence,Number,alzheimers_blood_based_biomarker_state
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,35,bbbm_testing,Male,0.0,Japan,alzheimers_blood_based_biomarker_state,Incidence,Number,alzheimers_blood_based_biomarker_state
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,2025,35,bbbm_testing_and_treatment,Female,0.0,Japan,alzheimers_blood_based_biomarker_state,Incidence,Number,alzheimers_blood_based_biomarker_state
...,...,...,...,...,...,...,...,...,...,...,...,...
3419995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,2100,249,bbbm_testing,Male,18657.0,Brazil,alzheimers_disease_state,Incidence,Number,alzheimers_disease_state
3419996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,2100,249,bbbm_testing_and_treatment,Female,18848.0,Brazil,alzheimers_mild_cognitive_impairment_state,Incidence,Number,alzheimers_mild_cognitive_impairment_state
3419997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,2100,249,bbbm_testing_and_treatment,Female,29047.0,Brazil,alzheimers_disease_state,Incidence,Number,alzheimers_disease_state
3419998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,2100,249,bbbm_testing_and_treatment,Male,13609.0,Brazil,alzheimers_mild_cognitive_impairment_state,Incidence,Number,alzheimers_mild_cognitive_impairment_state


In [23]:
with Timer():
    incidence_output = results.summarize_and_beautify(incidence_counts, 'to_state')
print_memory_usage(incidence_output)
print(len(incidence_output), 'rows')
incidence_output.tail()

Timer SummarizingTimer started
2026-02-17 17:07:53.628755
2026-02-17 17:08:31.873689
2026-02-17 17:17:42.523287
Elapsed time: 588.9133 seconds
Elapsed time: 0:09:48.913715
21.019239 MB 
656640 rows


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
656635,2100,United States of America,60_plus,Male,AD Dementia,BBBM Testing Only,Incidence,Rate,0.006270,0.005809,0.006822
656636,2100,United States of America,60_plus,Both,AD Dementia,BBBM Testing Only,Incidence,Rate,0.007874,0.007448,0.008285
656637,2100,United States of America,60_plus,Female,AD Dementia,BBBM Testing and Treatment,Incidence,Rate,0.009039,0.008436,0.009624
656638,2100,United States of America,60_plus,Male,AD Dementia,BBBM Testing and Treatment,Incidence,Rate,0.006138,0.005677,0.006695
656639,2100,United States of America,60_plus,Both,AD Dementia,BBBM Testing and Treatment,Incidence,Rate,0.007708,0.007266,0.008125


In [24]:
# Save incidence results
incidence_output.to_csv(output_dir / 'incidence.csv', index=False)

In [25]:
!date

Tue Feb 17 17:17:46 PST 2026
